# 安装google云盘必要的包

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
!nvidia-smi

Sun Dec 22 12:17:24 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

# 连接google云盘

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls drive/kaikeba/project01/code/

 05_seq2seq-Train.ipynb    main.py    README.md		 seq2seq_tf2
'beam search-Test.ipynb'   notebook   requirements.txt	 utils
 data			   pgn_tf2    result


# 载入代码

In [0]:
! pip install rouge
! pip install tensorflow-gpu==2.0.0

In [6]:
import tensorflow as tf
tf.__version__

'2.1.0-rc1'

In [0]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('drive/kaikeba/project01/code/')

# 导入训练代码

In [8]:
from seq2seq_tf2.train import train
from seq2seq_tf2.test import test
from utils.data_loader import build_dataset
from utils.config import train_data_path, test_data_path
from utils.config import train_x_seg_path, train_y_seg_path, test_x_seg_path, sample_total, batch_size, save_result_dir, \
    vocab_path,checkpoint_dir

Building prefix dict from the default dictionary ...
2019-12-22 12:21:48,767 : DEBUG : Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2019-12-22 12:21:49,447 : DEBUG : Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.741 seconds.
2019-12-22 12:21:49,513 : DEBUG : Loading model cost 0.741 seconds.
Prefix dict has been built succesfully.
2019-12-22 12:21:49,515 : DEBUG : Prefix dict has been built succesfully.


# 预处理数据

In [0]:
# build_dataset(train_data_path, test_data_path)

# 参数设置

In [12]:
!nvidia-smi

Sun Dec 22 12:24:07 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
batch_size=128

In [0]:
params = {"mode": 'train',
      "max_enc_len": 200,
      "max_dec_len": 34,
      "batch_size":batch_size ,
      "epochs": 25,
      "vocab_path": vocab_path,
      "learning_rate": 0.15,
      "adagrad_init_acc": 0.1,
      "rand_unif_init_mag": 0.02,
      "trunc_norm_init_std": 1e-4,
      "cov_loss_wt": 1.0,
      "max_grad_norm": 2.0,
      "vocab_size": 31820,
      "beam_size": batch_size,
      "embed_size": 300,
      "enc_units": 256,
      "dec_units": 256,
      "attn_units": 256,
      "train_seg_x_dir": train_x_seg_path,
      "train_seg_y_dir": train_y_seg_path,
      "test_seg_x_dir": test_x_seg_path,
      "checkpoints_save_steps": 5,
      "min_dec_steps": 4,
      "max_train_steps": sample_total // batch_size,
      "train_pickle_dir": '/opt/kaikeba/dataset/',
      "save_batch_train_data": False,
      "load_batch_train_data": False,
      "test_save_dir": save_result_dir,
      "pointer_gen": True,
      "use_coverage": True,
      "greedy_decode":False,
      "checkpoint_dir":checkpoint_dir}

# 训练模型

In [15]:
# GPU 700s TPU x s
train(params)

1 Physical GPUs, 1 Logical GPUs
Building the model ...
Epoch 1 Batch 0 Loss 3.9397
Epoch 1 Batch 50 Loss 2.5696
Epoch 1 Batch 100 Loss 2.9656
Epoch 1 Batch 150 Loss 3.0053
Epoch 1 Batch 200 Loss 2.7189
Epoch 1 Batch 250 Loss 2.7875
Epoch 1 Batch 300 Loss 2.9292
Epoch 1 Batch 350 Loss 2.9326
Epoch 1 Batch 400 Loss 2.7286
Epoch 1 Batch 450 Loss 2.9064
Epoch 1 Batch 500 Loss 3.0145
Epoch 1 Batch 550 Loss 2.9341
Epoch 1 Batch 600 Loss 2.9798
Epoch 1 Loss 2.9134
Time taken for 1 epoch 275.1854691505432 sec

Epoch 2 Batch 0 Loss 2.8372
Epoch 2 Batch 50 Loss 2.8802
Epoch 2 Batch 100 Loss 2.5456
Epoch 2 Batch 150 Loss 2.6823
Epoch 2 Batch 200 Loss 2.1952
Epoch 2 Batch 250 Loss 2.3244
Epoch 2 Batch 300 Loss 2.2058
Epoch 2 Batch 350 Loss 1.9894
Epoch 2 Batch 400 Loss 2.1620
Epoch 2 Batch 450 Loss 1.9597
Epoch 2 Batch 500 Loss 2.0120
Epoch 2 Batch 550 Loss 2.3145
Epoch 2 Batch 600 Loss 2.1027
Saving checkpoint for epoch 2 at /content/drive/kaikeba/project01/code/data/checkpoints/training_checkpoi

KeyboardInterrupt: ignored

In [0]:
test(params)